In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install noisereduce

In [ ]:
import librosa
import noisereduce as nr
import glob
#Just a sample of birds (from the scored_birds.json file)
bird_types = ['akiapo', 'aniani', 'apapan', 'barpet', 'crehon', 'elepai', 'ercfra', 'hawama',
              'hawcre', 'hawgoo', 'hawhaw', 'hawpet1', 'houfin', 'iiwi', 'jabwar', 'maupar',
              'omao', 'puaioh', 'skylar', 'warwhe1', 'yefcan']
data = []
labels = []
for i in range(len(bird_types)):
    count = 0
    for j in glob.glob('/kaggle/input/birdclef-2022/train_audio/'+bird_types[i]+'/*.ogg'):
        count += 1
        if (count > 10): 
            break
        y, sr = librosa.load(j)
        #Noise reduction
        reduced = nr.reduce_noise(y=y, sr=sr)
        dur = len(reduced) / sr
        #Split into 5 second intervals
        for k in range(int(dur/5)):
            data.append(reduced[5*k*sr:5*(k+1)*sr])
            labels.append(i)

In [ ]:

print(len(data))

In [ ]:
#FFT
ffts = []
sr = 22050
from scipy.fft import fft, fftfreq
for i in range(len(data)):
    yf = fft(data[i])
    xf = fftfreq(len(data[i]), 1.0/sr)
    print(xf[:int(len(yf)/2)])
    yf = yf[range(0, int(len(yf)/2), 50)]
    ffts.append(yf)

In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(4, 2)
for i in range(100,108):
    x = i % 4
    y = i // 4 - 25
    yf = ffts[i]
    xf = np.arange(0, 11030, 10)
    axs[x, y].plot(xf, np.abs(yf))

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
enc = OrdinalEncoder()
enc.fit_transform(np.array(labels).reshape(-1, 1))
fft_mags = np.abs(ffts)
X_train, X_test, y_train, y_test = train_test_split(
    fft_mags, labels, test_size=0.33)

In [ ]:
#SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score

clf = make_pipeline(StandardScaler(), LinearSVC(dual=False, C=0.1))
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
y_train_pred = clf.predict(X_train)
print('Train score:', f1_score(y_train, y_train_pred, average='micro'))
print('Test score:', f1_score(y_test, y_pred, average='micro'))